In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.image  as mpimg
import matplotlib.pyplot as plt
import os 
import zipfile
import random
import math
import datetime #timeline
import sys
import csv
import cv2


from IPython.display import clear_output

from sklearn.metrics import classification_report, confusion_matrix

from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard

from PIL import Image
from shutil import copyfile, rmtree
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model, optimizers, applications, models
from tensorflow.keras import regularizers, layers, losses, metrics
from tensorflow.keras import backend as K
from diagonalWeight import DiagonalWeight

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [ ]:
wd ='./work/predict'
ww = './weight_cut_data'
w1 = '1a'
w2 = '2a'
w3 = '3a'
w4 = '4a'
############ set up manual
num_model = 4
num_disease = 9
Input_Shape=(int(299*1.25),int(299*1.25),3)
Target_Size = Input_Shape[:-1]

create model 

In [ ]:
Pretrained_Model_1 = applications.inception_v3.InceptionV3(input_shape=Input_Shape, #note
                                                     include_top=False, 
                                                     input_tensor=None, #note
                                                     weights=None)
model_1 = models.Sequential()
model_1.add(Pretrained_Model_1)
model_1.add(layers.GlobalAveragePooling2D()) # note Flatten())
model_1.add(layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model_1.add(layers.Dropout(0.5))
model_1.add(layers.Dense(num_disease, activation='softmax', kernel_regularizer=regularizers.l2(0.01)))
model_1.load_weights(os.path.join(ww,w1,'intime_model_2020-03-28_20:10:01.h5'))

In [ ]:
Pretrained_Model_2 = applications.inception_v3.InceptionV3(input_shape=Input_Shape, #note
                                                     include_top=False, 
                                                     input_tensor=None, #note
                                                     weights=None)
model_2 = models.Sequential()
model_2.add(Pretrained_Model_2)
model_2.add(layers.GlobalAveragePooling2D()) # note Flatten())
model_2.add(layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model_2.add(layers.Dropout(0.5))
model_2.add(layers.Dense(num_disease, activation='softmax', kernel_regularizer=regularizers.l2(0.01)))
model_2.load_weights(os.path.join(ww,w2,'intime_model_2020-03-30_09:33:12.h5'))

In [ ]:
Pretrained_Model_3 = applications.inception_v3.InceptionV3(input_shape=Input_Shape, #note
                                                     include_top=False, 
                                                     input_tensor=None, #note
                                                     weights=None)
model_3 = models.Sequential()
model_3.add(Pretrained_Model_3)
model_3.add(layers.GlobalAveragePooling2D()) # note Flatten())
model_3.add(layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model_3.add(layers.Dropout(0.5))
model_3.add(layers.Dense(num_disease, activation='softmax', kernel_regularizer=regularizers.l2(0.01)))
model_3.load_weights(os.path.join(ww,w3,'Intime_mode_mix_b+o+cut_fold_3_checkpointv3_2-49-0.8568.hdf5'))

In [ ]:
Pretrained_Model_4 = applications.inception_v3.InceptionV3(input_shape=Input_Shape, #note
                                                     include_top=False, 
                                                     input_tensor=None, #note
                                                     weights=None)
model_4 = models.Sequential()
model_4.add(Pretrained_Model_4)
model_4.add(layers.GlobalAveragePooling2D()) # note Flatten())
model_4.add(layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model_4.add(layers.Dropout(0.5))
model_4.add(layers.Dense(num_disease, activation='softmax', kernel_regularizer=regularizers.l2(0.01)))
model_4.load_weights(os.path.join(ww,w4,'Intime_mode_mix_b+o+cut_fold_4_checkpointv3_2-48-0.8922.hdf5'))

In [ ]:
model_1_f2 = Model(inputs=model_1.input,outputs = model_1.output)
model_2_f2 = Model(inputs=model_2.input,outputs = model_2.output)
model_3_f2 = Model(inputs=model_3.input,outputs = model_3.output)
model_4_f2 = Model(inputs=model_4.input,outputs = model_4.output)

In [ ]:
model_1_f2.get_layer('inception_v3_input')._name = 'inception_v3_input_1'
model_2_f2.get_layer('inception_v3_input')._name = 'inception_v3_input_2'
model_3_f2.get_layer('inception_v3_input')._name = 'inception_v3_input_3'
model_4_f2.get_layer('inception_v3_input')._name = 'inception_v3_input_4'
model_1_f2.get_layer('inception_v3')._name = 'inception_v3_1'
model_2_f2.get_layer('inception_v3')._name = 'inception_v3_2'
model_3_f2.get_layer('inception_v3')._name = 'inception_v3_3'
model_4_f2.get_layer('inception_v3')._name = 'inception_v3_4'

In [ ]:
model_1_f2.summary()
model_2_f2.summary()
model_3_f2.summary()
model_4_f2.summary()

ensemble model

In [ ]:
f1 = layers.Reshape((model_1_f2.output.shape[1],1))(model_1_f2.output)
f2 = layers.Reshape((model_2_f2.output.shape[1],1))(model_2_f2.output)
f3 = layers.Reshape((model_3_f2.output.shape[1],1))(model_3_f2.output)
f4 = layers.Reshape((model_4_f2.output.shape[1],1))(model_4_f2.output)

merge = layers.concatenate([f1,f2,f3,f4],axis =2,name = 'concat')

f = layers.TimeDistributed(layers.Dense(num_model,use_bias=False
                 ,kernel_constraint=DiagonalWeight()
                ,kernel_initializer='identity'
                 ,kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-16, l2=1e-16)
                ),name = 'ensemble')(merge)
f = layers.Lambda(lambda x:K.sum(x, axis=2),name = 'combine')(f)
f = layers.Lambda(lambda x: x/K.sum(x),name = 'to_one')(f)
m = Model(inputs=[model_1_f2.input,model_2_f2.input,model_3_f2.input,model_4_f2.input],outputs = f)


In [ ]:
for layer in m.layers:
    layer.trainable=True
for layer in m.layers:
    if layer.name != 'ensemble':
        layer.trainable=False

In [ ]:
m.summary()

In [ ]:
model_checkpoint = ModelCheckpoint(os.path.join('./logs','Intime_cut_img_checkpointv5-{epoch:02d}-{acc:.4f}.hdf5')
                                   ,monitor='acc'
                                   ,mode = 'max', save_best_only=True, verbose=1)
callbacks = [model_checkpoint]

In [ ]:
Lr=0.01 #initial learning rate for Optimizer1, to be changed by lr_scheduler
Decay=0.0 # learning rate decay, not weight decay
Momentum=0.9
# Optimizer = optimizers.SGD(lr=Lr, decay=Decay, momentum=Momentum, clipnorm=5.0)
Optimizer = optimizers.Adam(1e-5)
Loss1='categorical_crossentropy'
Loss2='sparse_categorical_crossentropy'
Metrics='acc'
m.compile(optimizer=Optimizer, loss=Loss1, metrics=[Metrics])
# m.compile(optimizer=Optimizer, loss=Loss2, metrics=['sparse_categorical_accuracy'])

In [ ]:
x = []
y = []
classes= []
i = 0
for d in os.listdir(wd):
    print(d)
    for img in os.listdir(os.path.join(wd,d)):
        try:
            im = cv2.imread(os.path.join(wd,d,img))
            im = im/255.
            im = cv2.resize(im,Target_Size)
            im = im[:,:,[2,1,0]]
        except Exception as e:
            continue
        x.append(im)
        y_ = np.zeros(num_disease)
        y_[i]=1
        y.append(y_)
#         y.append([i])
        classes.append(i)
    i+=1
x =np.array(x)
y =np.array(y)

In [ ]:
m.fit(x =[x,x,x,x],y=y,epochs=200,verbose=2,batch_size = 24,class_weight='auto',callbacks=callbacks)

In [ ]:
Rescale=1./255
Rotation_Range=45
Width_Shift_Range=0.2
Height_Shift_Range=0.2
Shear_Range=0.2
Zoom_Range=0.2
Fill_Mode='constant'
stage_gen=ImageDataGenerator(
                            featurewise_center = True,# test
                            vertical_flip = True,# test
                            rotation_range=90, # test
                            rescale=Rescale,
#                             rotation_range=Rotation_Range,
                            width_shift_range=Width_Shift_Range,
                            height_shift_range=Height_Shift_Range,
                            shear_range=Shear_Range,
                            zoom_range=Zoom_Range,
                            horizontal_flip=True,
                            channel_shift_range=10,
                            fill_mode=Fill_Mode,
                            cval = 0
                            )
Class_Mode='categorical'
stage_generators2=stage_gen.flow_from_directory(wd,
                  interpolation='bicubic',
                  batch_size =1,
                  class_mode=Class_Mode,
                  shuffle=False,
                  target_size=Target_Size)
Y_pred_3 = m.predict([x,x,x,x])
y_pred_3 = np.argmax(Y_pred_3, axis=1)
print('Confusion Matrix')
print(confusion_matrix(stage_generators2.classes, y_pred_3))
print('Classification Report')
target_names = [cls for cls in stage_generators2.class_indices]
print(classification_report(classes, y_pred_3, target_names=target_names))

In [ ]:
y_p = []
x_p = []

for img in os.listdir(os.path.join('./work/predict','Balanitis')):
    try:
        im = cv2.imread(os.path.join(wd,'Balanitis',img))
        im = cv2.resize(im,Target_Size)
        im = im / 255
    except Exception as e:
        continue
    x_p.append(im)
    y_ = np.zeros(9,dtype = int)
    y_[0] =1
    y_p.append(y_)

y_p = np.array(y_p)
x_p = np.array(x_p)

In [ ]:
Rescale=1./255
Rotation_Range=45
Width_Shift_Range=0.2
Height_Shift_Range=0.2
Shear_Range=0.2
Zoom_Range=0.2
Fill_Mode='constant'
stage_gen=ImageDataGenerator(
                            featurewise_center = True,# test
                            vertical_flip = True,# test
                            rotation_range=90, # test
                            rescale=Rescale,
#                             rotation_range=Rotation_Range,
                            width_shift_range=Width_Shift_Range,
                            height_shift_range=Height_Shift_Range,
                            shear_range=Shear_Range,
                            zoom_range=Zoom_Range,
                            horizontal_flip=True,
                            channel_shift_range=10,
                            fill_mode=Fill_Mode,
                            cval = 0
                            )
Class_Mode='categorical'
stage_generators2=stage_gen.flow_from_directory(wd,
                  interpolation='bicubic',
                  batch_size =1,
                  class_mode=Class_Mode,
                  shuffle=False,
                  target_size=Target_Size)
Y_pred_3 = model_1.predict_generator(stage_generators2)
y_pred_3 = np.argmax(Y_pred_3, axis=1)
print('Confusion Matrix')
print(confusion_matrix(stage_generators2.classes, y_pred_3))
print('Classification Report')
target_names = [cls for cls in stage_generators2.class_indices]
print(classification_report(stage_generators2.classes, y_pred_3, target_names=target_names))